In [129]:
#balances the number of No Claims in training set
#according to % in Models.csv
import pandas as pd
import numpy as np

In [130]:
#data
dataset = pd.read_csv('/home/kate/data/ClaimPrediction/fdata_v1_encd.csv', index_col=None)
target_column = 'hasclaim'

In [131]:
BaseModel=[ 'accidentpreventioncourseind_encd',
 'carpoolind_encd',
 'classcd_encd',
 'driverage',
 'drivernumber',
 'driverstatuscd_encd',
 'drivertrainingind_encd',
 'estimatedannualdistance',
 'gooddriverind_encd',
 'maturedriverind_encd',
 'mvrstatus_encd',
 'mvrstatusage',
 'ratingvalue',
 'relationshiptoinsuredcd_encd',
 'scholasticdiscountind_encd',
 'vehbodytypecd_encd',
 'vehicleage',
 'vehnumber',
 'gendercd_encd',
 'maritalstatuscd_encd',
 'enginecylinders_encd',
 'passiveseatbeltind_encd',
 'acci_pointschargedterm',
 'acci_last_infractionage']
ShortBestModel2 = [ 'accidentpreventioncourseind_encd',
 'carpoolind_encd',
 'classcd_encd',
 'driverage',
 'drivernumber',
 'driverstatuscd_encd',
 'drivertrainingind_encd',
 'estimatedannualdistance',
 'gooddriverind_encd',
 'maturedriverind_encd',
 'mvrstatus_encd',
 'mvrstatusage',
 'ratingvalue',
 'relationshiptoinsuredcd_encd',
 'scholasticdiscountind_encd',
 'vehbodytypecd_encd',
 'vehicleage',
 'vehnumber',
 'gendercd_encd',
 'maritalstatuscd_encd',
 'enginecylinders_encd',
 'passiveseatbeltind_encd']

In [132]:
#models and analyzing results directory
ModelsDir='/home/kate/data/ClaimPrediction/r11_add/'

In [133]:
#comparing model metrics with t-test
#and save results
import scipy.stats as stats
def AnalyzeAndSaveModelsResults(result_df,result_lst,ModelName,filename):
    df=pd.DataFrame([result_lst])
    TestSizeColumns=['S0.45','S0.4','S0.35','S0.3','S0.25','S0.2','S0.15','S0.1']
    #TestSizeColumns=['S0.2','S0.15','S0.1']
    df.columns=['Model']+TestSizeColumns
    #mean
    df['Mean'] = df.drop('Model', axis=1).mean(axis=1)
    df['t-pvalue'] = 1
    df['t-statistic'] = 0
    df['Group'] = 1
    #t-test with base model
    if ModelName!='BaseModel':
        base_model=result_df[result_df['Model'] == 'BaseModel'].iloc[0]
        current_model=df.iloc[0]
        t=stats.ttest_ind(base_model[TestSizeColumns].tolist(),current_model[TestSizeColumns].tolist())
        line_to_save=[current_model['Model']]
        line_to_save.extend(current_model[TestSizeColumns].tolist())
        line_to_save.append(current_model[TestSizeColumns].mean())
        line_to_save.append(t.pvalue)
        line_to_save.append(t.statistic)
        if ((t.pvalue<=0.05) and (base_model['Mean']<current_model['Mean'])):
            line_to_save.append(2)
        elif ((t.pvalue<=0.05) and (base_model['Mean']>current_model['Mean'])):
            line_to_save.append(3)    
        else:
            line_to_save.append(1)                  
        df_to_save=pd.DataFrame([line_to_save])
        df_to_save.columns=['Model']+TestSizeColumns+['Mean','t-pvalue','t-statistic','Group']
        result_df=result_df.append(df_to_save, ignore_index=True)
    else:
        result_df=result_df.append(df, ignore_index=True)
    result_df.to_csv(ModelsDir+filename, index = False)
    return result_df

In [135]:
#xgb library and parameters to tune later
import xgboost as xgb
#Adjusted Optimized
xgb_params = {
 'booster': 'gbtree',
 'cosample_bytree': 0.9786846475506199,
 'max_depth': 4,
 'subsample': 0.76097593937313557,
 'colsample_bylevel': 0.23209450560452577,
 'eta': 0.012080410469107298,
 'eval_metric': 'auc',
 'silent': True,
 'objective': 'binary:logistic',
 'seed': 42}

In [136]:
#Evaluation metric to be used in tuning
from sklearn.metrics import roc_auc_score
def gini(y, pred):
    g = np.asarray(np.c_[y, pred, np.arange(len(y)) ], dtype=np.float)
    g = g[np.lexsort((g[:,2], -1*g[:,1]))]
    gs = g[:,0].cumsum().sum() / g[:,0].sum()
    gs -= (len(y) + 1) / 2.
    return gs / len(y)
def gini_xgb(pred, y):
    y = y.get_label()
    return 'gini', gini(y, pred) / gini(y, y)

In [137]:
#Random Undersampler to balance the dataset
from imblearn.under_sampling import RandomUnderSampler
#ratio=0.918 to tune later

In [138]:
#StratifiedKFold
from sklearn.model_selection import StratifiedKFold
nrounds=5000 # need to change to 2000
kfold = 10  # need to change to 10
skf = StratifiedKFold(n_splits=kfold, random_state=42)

In [139]:
#splitting to train/test in the loop below
from sklearn.model_selection import train_test_split

In [140]:
#each line in the file contains the model name and set of features to analize
models = pd.read_csv(ModelsDir+'Models.csv', index_col=None)

In [141]:
#summary for test and train metrics for each model to test overfitting
models_test_gini_df=pd.DataFrame()
models_test_roc_auc_df=pd.DataFrame()
#
models_train_gini_df=pd.DataFrame()
models_train_roc_auc_df=pd.DataFrame()

In [142]:
for index, row in models.iterrows():
    #for test and train metrics for each model to test overfitting
    gini_test_lst=[]
    roc_auc_test_lst=[]
    gini_train_lst=[]
    roc_auc_train_lst=[]
    #Starting analyzing metric
    print (index, ': Analyzing model %s'%row['Model'])
    gini_test_lst.append(row['Model'])
    roc_auc_test_lst.append(row['Model'])
    gini_train_lst.append(row['Model'])
    roc_auc_train_lst.append(row['Model'])
    Ratio=row['Ratio']
    #calculating metrics for the current RandomUnderSample Ratio and 
    #several data sizes
    for s in (0.45,0.4,0.35,0.3,0.25,0.2,0.15,0.1):
        print ('Test size %s'%s)
        X, X_test, y, y_test = train_test_split(dataset.loc[:,BaseModel], dataset[target_column], test_size=s, random_state=42)
        #prediction dataframes
        y_pred_test=pd.DataFrame(index=y_test.index)
        y_pred_test[target_column]=0
        y_pred_train=pd.DataFrame(index=y.index)
        y_pred_train[target_column]=0
        #
        X = X.values
        y = y.values
        #Stratified Fold
        for i, (train_index, test_index) in enumerate(skf.split(X, y)):
            print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
            #getting fold data
            X_train, X_valid = X[train_index], X[test_index]
            y_train, y_valid = y[train_index], y[test_index]
            HasClaim=len(y_train[y_train==1])
            Total_train=len(y_train)
            NoClaim_train=len(y_train[y_train==0])
            #balancing dataset
            #converting Ratio to HasClaim=0 Size
            NoClaim=int(HasClaim*Ratio/(1-Ratio))
            print('--------------------------------------------------------------------')
            print('Ratio: %s'%Ratio)
            print('Total: %s'%Total_train)
            print('HasClaim Size: %s'%HasClaim)
            print('NoClaim Train Size: %s'%NoClaim_train)
            print('Calculated NoClaim Size: %s'%NoClaim)
            print('--------------------------------------------------------------------')
            NoClaimRatio={0 : NoClaim}
            rus = RandomUnderSampler(random_state=42,ratio=NoClaimRatio)
            X_res, y_res = rus.fit_sample(X_train, y_train)
            print('--------------------------------------------------------------------')
            Total_balnced=len(y_res)
            HasClaim_balanced=len(y_res[y_res==1])
            NoClaim_balanced=len(y_res[y_res==0])
            print('Total Balanced: %s'%Total_balnced)
            print('HasClaim Balanced Size: %s'%HasClaim_balanced)
            print('NoClaim Balanced Size: %s'%NoClaim_balanced)
            print('--------------------------------------------------------------------')
            #applying XGB
            d_train = xgb.DMatrix(X_res, y_res) 
            d_valid = xgb.DMatrix(X_valid, y_valid) 
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]
            xgb_model = xgb.train(xgb_params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=1000)
            y_pred_test[target_column] +=  xgb_model.predict(xgb.DMatrix(X_test.values), ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
            y_pred_train[target_column] += xgb_model.predict(xgb.DMatrix(X), ntree_limit=xgb_model.best_ntree_limit+50) / (kfold)
        #Prediction results
        #test
        print('--------------------------------------------------------------------')
        print(row['Model'])
        print('Test Size %f'%s)
        print('Ratio: %s'%Ratio)
        print('--------------------------------------------------------------------')
        g=gini(y_test,y_pred_test)/gini(y_test,y_test)
        print('Test Gini - %f'%g)
        gini_test_lst.append(g)
        ROC_AUC=roc_auc_score(y_test, y_pred_test)
        print('Test ROC_AUC - %f'%ROC_AUC)
        roc_auc_test_lst.append(ROC_AUC)
        #train
        g=gini(y,y_pred_train)/gini(y,y)
        print('Train Gini - %f'%g)
        gini_train_lst.append(g)
        ROC_AUC=roc_auc_score(y, y_pred_train)
        print('Train ROC_AUC - %f'%ROC_AUC)
        roc_auc_train_lst.append(ROC_AUC)
    #save model analysis results
    models_test_gini_df=AnalyzeAndSaveModelsResults(models_test_gini_df,gini_test_lst,row['Model'],'models_test_gini.csv')
    models_test_roc_auc_df=AnalyzeAndSaveModelsResults(models_test_roc_auc_df,roc_auc_test_lst,row['Model'],'models_test_roc_auc.csv')
    models_train_gini_df=AnalyzeAndSaveModelsResults(models_train_gini_df,gini_train_lst,row['Model'],'models_train_gini.csv')
    models_train_roc_auc_df=AnalyzeAndSaveModelsResults(models_train_roc_auc_df,roc_auc_train_lst,row['Model'],'models_train_roc_auc.csv')

0 : Analyzing model BaseModel
Test size 0.45
 xgb kfold: 1  of  10 : 
--------------------------------------------------------------------
Ratio: 0.5
Total: 65771
HasClaim Size: 3448
NoClaim Train Size: 62323
Calculated NoClaim Size: 3448
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 6896
HasClaim Balanced Size: 3448
NoClaim Balanced Size: 3448
--------------------------------------------------------------------
[0]	train-auc:0.616824	valid-auc:0.61396	train-gini:0.009222	valid-gini:0.227706
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
Stopping. Best iteration:
[704]	train-auc:0.769519	valid-auc:0.706058	train-gini:0.539037	valid-gini:0.412115

 xgb kfold: 2  of  10 : 
--------------------------------------------------------------------
Ratio: 0.5
Total: 65771
HasClaim Size: 3448


Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
Stopping. Best iteration:
[719]	train-auc:0.77076	valid-auc:0.694045	train-gini:0.541518	valid-gini:0.38809

--------------------------------------------------------------------
BaseModel
Test Size 0.450000
Ratio: 0.5
--------------------------------------------------------------------
Test Gini - 0.411702
Test ROC_AUC - 0.705851
Train Gini - 0.481755
Train ROC_AUC - 0.740877
Test size 0.4
 xgb kfold: 1  of  10 : 
--------------------------------------------------------------------
Ratio: 0.5
Total: 71751
HasClaim Size: 3735
NoClaim Train Size: 68016
Calculated NoClaim Size: 3735
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 7470
HasClaim Balanced Size: 3735
NoClaim Balanced Size: 3735
---------------------------------------------------

 xgb kfold: 10  of  10 : 
--------------------------------------------------------------------
Ratio: 0.5
Total: 71752
HasClaim Size: 3735
NoClaim Train Size: 68017
Calculated NoClaim Size: 3735
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 7470
HasClaim Balanced Size: 3735
NoClaim Balanced Size: 3735
--------------------------------------------------------------------
[0]	train-auc:0.626763	valid-auc:0.604004	train-gini:0.075192	valid-gini:0.219938
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
Stopping. Best iteration:
[637]	train-auc:0.761443	valid-auc:0.689132	train-gini:0.522884	valid-gini:0.378263

--------------------------------------------------------------------
BaseModel
Test Size 0.400000
Ratio: 0.5
--------------------------------------------------------------------
Tes

Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
Stopping. Best iteration:
[633]	train-auc:0.75359	valid-auc:0.699651	train-gini:0.507179	valid-gini:0.399302

 xgb kfold: 10  of  10 : 
--------------------------------------------------------------------
Ratio: 0.5
Total: 77732
HasClaim Size: 4042
NoClaim Train Size: 73690
Calculated NoClaim Size: 4042
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 8084
HasClaim Balanced Size: 4042
NoClaim Balanced Size: 4042
--------------------------------------------------------------------
[0]	train-auc:0.617112	valid-auc:0.612143	train-gini:0.005732	valid-gini:0.234651
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[1000]	train-auc:0.77046	valid-auc:0

Stopping. Best iteration:
[862]	train-auc:0.765693	valid-auc:0.706652	train-gini:0.531385	valid-gini:0.413302

 xgb kfold: 9  of  10 : 
--------------------------------------------------------------------
Ratio: 0.5
Total: 83711
HasClaim Size: 4355
NoClaim Train Size: 79356
Calculated NoClaim Size: 4355
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 8710
HasClaim Balanced Size: 4355
NoClaim Balanced Size: 4355
--------------------------------------------------------------------
[0]	train-auc:0.63384	valid-auc:0.6106	train-gini:0.125878	valid-gini:0.234636
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[1000]	train-auc:0.771928	valid-auc:0.693939	train-gini:0.543855	valid-gini:0.387877
Stopping. Best iteration:
[1155]	train-auc:0.777916	valid-auc:0.694602	train-gini:0.555831	valid-gin

...

Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[1000]	train-auc:0.737624	valid-auc:0.713107	train-gini:0.475246	valid-gini:0.426215
[2000]	train-auc:0.759567	valid-auc:0.718446	train-gini:0.519132	valid-gini:0.436893
Stopping. Best iteration:
[1967]	train-auc:0.75902	valid-auc:0.718534	train-gini:0.518038	valid-gini:0.437069

 xgb kfold: 3  of  10 : 
--------------------------------------------------------------------
Ratio: 0.94
Total: 95668
HasClaim Size: 5025
NoClaim Train Size: 90643
Calculated NoClaim Size: 78724
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 83749
HasClaim Balanced Size: 5025
NoClaim Balanced Size: 78724
--------------------------------------------------------------------
[0]	train-auc:0.621945	valid-auc:0.611493	train-gini:0.042118	valid-gini:0.246381
Multiple

[0]	train-auc:0.615806	valid-auc:0.623243	train-gini:0.040755	valid-gini:0.226438
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[1000]	train-auc:0.737878	valid-auc:0.711067	train-gini:0.475755	valid-gini:0.422133
[2000]	train-auc:0.758997	valid-auc:0.714722	train-gini:0.517993	valid-gini:0.429444
Stopping. Best iteration:
[1918]	train-auc:0.75749	valid-auc:0.714962	train-gini:0.514979	valid-gini:0.429924

 xgb kfold: 2  of  10 : 
--------------------------------------------------------------------
Ratio: 0.94
Total: 101647
HasClaim Size: 5315
NoClaim Train Size: 96332
Calculated NoClaim Size: 83268
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 88583
HasClaim Balanced Size: 5315
NoClaim Balanced Size: 83268
--------------------------------------------------------------------
[0]	tra

Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[1000]	train-auc:0.738694	valid-auc:0.707763	train-gini:0.477388	valid-gini:0.415526
Stopping. Best iteration:
[1365]	train-auc:0.747759	valid-auc:0.711046	train-gini:0.495516	valid-gini:0.422091

--------------------------------------------------------------------
Model0.94
Test Size 0.150000
Ratio: 0.94
--------------------------------------------------------------------
Test Gini - 0.425925
Test ROC_AUC - 0.712963
Train Gini - 0.502273
Train ROC_AUC - 0.751136
Test size 0.1
 xgb kfold: 1  of  10 : 
--------------------------------------------------------------------
Ratio: 0.94
Total: 107627
HasClaim Size: 5615
NoClaim Train Size: 102012
Calculated NoClaim Size: 87968
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 93583
HasClaim Balan

 xgb kfold: 9  of  10 : 
--------------------------------------------------------------------
Ratio: 0.94
Total: 107628
HasClaim Size: 5615
NoClaim Train Size: 102013
Calculated NoClaim Size: 87968
--------------------------------------------------------------------
--------------------------------------------------------------------
Total Balanced: 93583
HasClaim Balanced Size: 5615
NoClaim Balanced Size: 87968
--------------------------------------------------------------------
[0]	train-auc:0.618674	valid-auc:0.613952	train-gini:0.054663	valid-gini:0.240682
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[1000]	train-auc:0.738138	valid-auc:0.701283	train-gini:0.476275	valid-gini:0.402565
Stopping. Best iteration:
[1091]	train-auc:0.740346	valid-auc:0.702166	train-gini:0.48069	valid-gini:0.404331

 xgb kfold: 10  of  10 : 
-----------------------------------------------------------------

ValueError: With under-sampling methods, the number of samples in a class should be less or equal to the original number of samples. Originally, there is 62323 samples and 65511 samples are asked.

Red and Green bars below indicates models where mean of gini is likely different from a previous model

In [ ]:
models_test_gini_df

In [ ]:
models_test_gini_df.set_index('Model', inplace=True)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
colors = {1: 'blue', 2: 'green',3: 'red'}

In [ ]:
customcmap = tuple(models_test_gini_df['Group'].map(colors))
models_test_gini_df['Mean'].plot(kind='barh',  color=[customcmap], figsize=(10, 12))